Competition on Kaggle: https://www.kaggle.com/c/how-good-is-your-medium-article/leaderboard

In [2]:
!pip install tqdm

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [115]:
import os
import codecs
import json
import seaborn as sns
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack

from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from datetime import date

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [5]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

In [55]:
def extract_features_and_write(path_to_data,
                               inp_filename, is_train=True):
    ident = 0
    features = ['_id', 'content', 'published', 'title', 'author']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding="utf-8")
                     for feat in features]
 
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:
    
        for line in tqdm_notebook(inp_json_file):
            dict_data = {}
            json_data = read_json_line(line)
            for i in range(len(feature_files)):
                dict_data[ident] = json_data[features[i]]
                feature_files[i].write(str(dict_data) + '\n')
            ident += 1
    for i in feature_files:
        i.close()

In [56]:
PATH_TO_DATA = 'D:\mlcourse\Databases\kaggle_medium'

In [57]:
extract_features_and_write(PATH_TO_DATA, 'train.json', is_train=True)

In [38]:
extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

## Add the following groups of features:

- Tf-Idf with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Tf-Idf with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Time features: publication hour, whether it's morning, day, night, whether it's a weekend
- Bag of authors (i.e. One-Hot-Encoded author names)

#### Extacting data

In [101]:
prefix = 'train'
features = ['_id', 'content', 'published', 'title', 'author']

l_files = [os.path.join(PATH_TO_DATA,
                                       '{}_{}.txt'.format(prefix, feat))   
                     for feat in features]

In [102]:
%%time
df_all_data = pd.DataFrame({})
for k in range(0, len(features)):
    f = codecs.open(l_files[k], "r", "utf-8")
    arr = []
    count = 0
    for line in f:
        count += 1
        arr.append(list(eval(line[:-2]).values())[0])
    f.close()
    ser_buf = pd.Series(arr)
    df_all_data[features[k]] = ser_buf

Wall time: 1h 31min 57s


In [103]:
df_all_data['content'] = df_all_data['content'].map(strip_tags)    

Adding a popularity column

In [104]:
data = pd.read_csv('D:\mlcourse\Databases\kaggle_medium/train_log1p_recommends.csv', sep=',')

In [105]:
df_all_data = df_all_data.join(data)

## Strip content: \xa0, \u200b

## Division publications by hour, whether it's morning, day, night, whether it's a weekend


In [106]:
for i in df.index:
    df_all_data['published'][i] = list(df_all_data['published'].values[i].values())[0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Creating the columns date and time and preprocessing author

In [107]:
def pub_to_time(publicate):
    return publicate[11:-5]

def pub_to_date(publicate):
    return publicate[:10] 

def time_to_sec(time_s):
    return ((int(time_s[:2])* 60 * 60) + (int(time_s[3:5])) * 60 + (int(time_s[6:])))

# may be needed other division
def sec_to_tofd(time_sec):
    x = 3600
    if(time_sec > 18 * x):
        return 'E'
    elif(time_sec > 12 * x):
        return 'D'
    elif(time_sec > 6 * x):
        return 'M'
    else:
        return 'N'
    
def day_of_week(d):
    days = date(int(d[:4]), int(d[5:7]), int(d[8:]))
    return(days.weekday())

def is_weekend(d):
    if (d > 4):
        return True
    return False

def rewrite_author(author):
    return author['twitter']

In [108]:
df_all_data['date'] = df_all_data['published'].map(pub_to_date)
df_all_data['time'] = df_all_data['published'].map(pub_to_time)
df_all_data['time_sec'] = df_all_data['time'].map(time_to_sec)
df_all_data['time_of_day'] = df_all_data['time_sec'].map(sec_to_tofd)
df_all_data['week_day'] = df_all_data['date'].map(day_of_week)
df_all_data['weekend'] = df_all_data['week_day'].map(is_weekend)
df_all_data['twitter'] = df_all_data['author'].map(rewrite_author)

In [109]:
df_all_data.head(2)

,_id,content,published,title,author,id,log_recommends,date,time,time_sec,time_of_day,week_day,weekend,twitter
0,https://medium.com/policy/medium-terms-of-serv...,"Medium Everyone’s stories and ideas Aug 13, 20...",2012-08-13T22:54:53.510Z,Medium Terms of Service – Medium Policy – Medium,"{'name': None, 'url': 'https://medium.com/@Med...",8,9.01201,2012-08-13,22:54:53,82493,E,0,False,@Medium
1,https://medium.com/policy/amendment-to-medium-...,"Medium Everyone’s stories and ideas Aug 2, 201...",2015-08-03T07:44:50.331Z,Amendment to Medium Terms of Service Applicabl...,"{'name': None, 'url': 'https://medium.com/@Med...",14,3.49651,2015-08-03,07:44:50,27890,M,0,False,@Medium


In [117]:
# Not here
feats = ['content', 'title', 'time_of_day', 'week_day', 'weekend', 'twitter', 'log_recommends']
df = df_all_data[feats]
X_train, y_train = train_test_split(df, random_state=17)

## Creating Tf-Idf 

- with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)


In [134]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_title = count_vect.fit_transform(df.title)

tf_transformer_title = TfidfTransformer(use_idf=False).fit(df_counts_title)
df_tf_title = tf_transformer_title.transform(df_counts_title)

tfidf_transformer = TfidfTransformer()
df_tfidf_title = tfidf_transformer.fit_transform(df_counts_title)
df_tfidf_title.shape

Wall time: 6.38 s


In [135]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_content = count_vect.fit_transform(df.content)

tf_transformer_content = TfidfTransformer(use_idf=False).fit(df_counts_content)
df_tf_content = tf_transformer_content.transform(df_counts_content)

tfidf_transformer = TfidfTransformer()
df_tfidf_content = tfidf_transformer.fit_transform(df_counts_content)
df_tfidf_content.shape

Wall time: 1h 27min 30s
Parser   : 191 ms

In [136]:
df_extended = df.join(df_tfidf_content).join(df_tfidf_title)

AttributeError: index not found

In [ ]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))
    
text_clf = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)
twenty_test = fetch_20newsgroups(subset='test',
     categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)    

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),
    ])
_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)